In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import glob as gb
import cv2
import PIL
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from keras.models import Model

In [13]:
trainpath = '/kaggle/input/riceleafdiseasedataset/dataset/train'
testpath = '/kaggle/input/riceleafdiseasedataset/dataset/test'

# Image Processing

In [14]:
# Image Processing - Training Data
new_size = 224
train_images = []
train_labels = []
class_disease = {'BacterialBlight': 0, 'Blast': 1, 'BrownSpot': 2, 'Tungro': 3}

for i in os.listdir(trainpath):
    if i in class_disease:
        print("Entering the folder:", i)
        files = gb.glob(pathname=str(trainpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(trainpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder:", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            train_images.append(list(resize_image))
            train_labels.append(class_disease[i])

# Image Processing - Testing Data
new_size = 224
test_images = []
test_labels = []

for i in os.listdir(testpath):
    if i in class_disease:
        print("Entering to the folder name:", i)
        files = gb.glob(pathname=str(testpath + '/' + i + '/*.jpg')) + gb.glob(pathname=str(testpath + '/' + i + '/*.JPG'))
        print("Number of images in the folder is", len(files))
        for j in files:
            image_raw = cv2.imread(j)
            image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
            resize_image = cv2.resize(image, (new_size, new_size))
            test_images.append(list(resize_image))
            test_labels.append(class_disease[i])

def list_to_array_train(train_images, train_labels):
    return np.array(train_images), np.array(train_labels)

X_train, y_train = list_to_array_train(train_images, train_labels)

def list_to_array_test(test_images, test_labels):
    return np.array(test_images), np.array(test_labels)

X_test, y_test = list_to_array_test(test_images, test_labels)

print(X_train.shape)
print("*" * 20)
print(y_train.shape)
print("*" * 20)
print(X_test.shape)
print(y_test.shape)

def keras_to_categorical(y_train, y_test):
    return to_categorical(y_train), to_categorical(y_test)

y_train1 = y_train
y_test1 = y_test
y_train, y_test = keras_to_categorical(y_train, y_test)

y_train1.shape, y_test1.shape

def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

gpus = tf.config.list_physical_devices('GPU')
print(f"Num GPUs Available: {len(gpus)}")

if len(gpus) < 2:
    print("Not enough GPUs available, ensure your environment is configured correctly")
else:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.MirroredStrategy()

Entering the folder: Tungro
Number of images in the folder: 1108
Entering the folder: BacterialBlight
Number of images in the folder: 1384
Entering the folder: Blast
Number of images in the folder: 1240
Entering the folder: BrownSpot
Number of images in the folder: 1400
Entering to the folder name: Tungro
Number of images in the folder is 200
Entering to the folder name: BacterialBlight
Number of images in the folder is 200
Entering to the folder name: Blast
Number of images in the folder is 200
Entering to the folder name: BrownSpot
Number of images in the folder is 200
(5132, 224, 224, 3)
********************
(5132,)
********************
(800, 224, 224, 3)
(800,)
Num GPUs Available: 2


In [15]:
# Function to convert one-hot encoded labels to categorical labels
def convert_one_hot_to_categorical(one_hot_labels):
    return np.argmax(one_hot_labels, axis=1)

In [16]:
# Deep Feature Extraction - EfficientNet
def model_effnet():
    Effnet_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in Effnet_model.layers:
        layer.trainable = False
    feature = GlobalAveragePooling2D()(Effnet_model.output)
    output = Model(inputs=Effnet_model.input, outputs=feature)
    return output


In [17]:
# LBP

from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from tqdm import tqdm

# Parameters for LBP
radius = 3
n_points = 8 * radius

def compute_lbp_features(image):
    if image.ndim == 3 and image.shape[2] == 3:  # Check if the image is RGB
        gray_image = rgb2gray(image)
    elif image.ndim == 2:  # If the image is already grayscale
        gray_image = image
    else:
        raise ValueError(f"Unexpected image shape: {image.shape}")
    
    lbp = local_binary_pattern(gray_image, n_points, radius, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalize the histogram
    return hist

def extract_lbp_features(X):
    return np.array([compute_lbp_features(img) for img in X])

train_feature_lbp = extract_lbp_features(X_train)
test_feature_lbp = extract_lbp_features(X_test)


/opt/conda/lib/python3.10/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


# EfficientNetB3 + LBP + Random Forest Classifier

In [18]:
from sklearn.ensemble import RandomForestClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        model_FE_effnet = model_effnet()
        model_FE_effnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_effnet = model_FE_effnet.predict(X_train)
        test_feature_effnet = model_FE_effnet.predict(X_test)
        
        final_train = np.hstack((train_feature_effnet, train_feature_lbp))
        final_test = np.hstack((test_feature_effnet, test_feature_lbp))

        rf = RandomForestClassifier()
        rf = rf.fit(final_train, y_train)
        test_pred = rf.predict(final_test)

        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in tqdm(range(num_runs), desc="Evaluating EfficientNetB3+LBP: "):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Calculate average metrics
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

# Print average metrics
print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'EfficientNetB3+LBP+RF.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


Evaluating EfficientNetB3+LBP:   0%|          | 0/10 [00:00<?, ?it/s]

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 34s 142ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step


Evaluating EfficientNetB3+LBP:  10%|█         | 1/10 [01:04<09:39, 64.36s/it]

Run 1 - Accuracy: 0.97250000, Recall: 0.97250000, Precision: 0.99747475, F1 Score: 0.98472220, AUC: 0.98583333
161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 131ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step


Evaluating EfficientNetB3+LBP:  20%|██        | 2/10 [02:02<08:04, 60.57s/it]

...

Run 8 - Accuracy: 0.97375000, Recall: 0.97375000, Precision: 0.99873096, F1 Score: 0.98600746, AUC: 0.98666667
161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 128ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step


Evaluating EfficientNetB3+LBP:  90%|█████████ | 9/10 [08:51<00:58, 58.76s/it]

Run 9 - Accuracy: 0.97125000, Recall: 0.97125000, Precision: 1.00000000, F1 Score: 0.98530330, AUC: 0.98562500
161/161 ━━━━━━━━━━━━━━━━━━━━ 31s 142ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step


Evaluating EfficientNetB3+LBP: 100%|██████████| 10/10 [09:51<00:00, 59.12s/it]

Run 10 - Accuracy: 0.96500000, Recall: 0.96500000, Precision: 0.99747475, F1 Score: 0.98076229, AUC: 0.98208333

Average Metrics:
Accuracy: 0.97387500 (std: 0.00372701)
Recall: 0.97387500 (std: 0.00372701)
Precision: 0.99860919 (std: 0.00119232)
F1: 0.98599651 (std: 0.00224662)
Auc: 0.98670833 (std: 0.00194945)

Metrics results saved to 'EfficientNetB3+LBP+RF.xlsx'


# EfficientNetB3 + LBP + Decision Tree Classifier

In [19]:
from sklearn.tree import DecisionTreeClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        model_FE_effnet = model_effnet()
        model_FE_effnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_effnet = model_FE_effnet.predict(X_train)
        test_feature_effnet = model_FE_effnet.predict(X_test)
        
        final_train = np.hstack((train_feature_effnet, train_feature_lbp))
        final_test = np.hstack((test_feature_effnet, test_feature_lbp))

        # Define and train Decision Tree Classifier
        dt = DecisionTreeClassifier()
        dt.fit(final_train, y_train)
        test_pred = dt.predict(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'EfficientNetB3+LBP+DTC.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 128ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 1 - Accuracy: 0.90125000, Recall: 0.90125000, Precision: 0.90410020, F1 Score: 0.89964353, AUC: 0.93416667
161/161 ━━━━━━━━━━━━━━━━━━━━ 28s 127ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 2 - Accuracy: 0.90875000, Recall: 0.90875000, Precision: 0.91119120, F1 Score: 0.90724840, AUC: 0.93916667
161/161 ━━━━━━━━━━━━━━━━━━━━ 32s 130ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step
Run 3 - Accuracy: 0.90625000, Recall: 0.90625000, Precision: 0.90808482, F1 Score: 0.90487566, AUC: 0.93750000
161/161 ━━━━━━━━━━━━━━━━━━━━ 28s 128ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 4 - Accuracy: 0.91500000, Recall: 0.91500000, Precision: 0.91839840, F1 Score: 0.91426906, AUC: 0.94333333
161/161 ━━━━━━━━━━━━━━━━━━━━ 28s 125ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 5 - Accuracy: 0.89875000, Recall: 0.89875000, Precision: 0.90042804, F1 Score: 0.89766864, AUC: 0.93250000
161/161 ━━━━━━━━━━━━━━━━━

# EfficientNetB3 + LBP + KNN Classifier

In [20]:
from sklearn.neighbors import KNeighborsClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test, num):
    with strategy.scope():  
        model_FE_effnet = model_effnet()
        model_FE_effnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_effnet = model_FE_effnet.predict(X_train)
        test_feature_effnet = model_FE_effnet.predict(X_test)
        
        final_train = np.hstack((train_feature_effnet, train_feature_lbp))
        final_test = np.hstack((test_feature_effnet, test_feature_lbp))

        # Define and train KNN Classifier
        knn = KNeighborsClassifier(n_neighbors=num)
        knn.fit(final_train, y_train)
        test_pred = knn.predict(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, test_pred)
        recall = recall_score(y_test, test_pred, average='weighted')
        precision = precision_score(y_test, test_pred, average='weighted')
        f1 = f1_score(y_test, test_pred, average='weighted')
        auc = roc_auc_score(y_test, test_pred, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}
num = 5

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test, num)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")
    num = num+1

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'EfficientNetB3+LBP+KNN.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 130ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 1 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 0.99833333
161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 127ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 2 - Accuracy: 0.99125000, Recall: 0.99125000, Precision: 0.99750000, F1 Score: 0.99435455, AUC: 0.99520833
161/161 ━━━━━━━━━━━━━━━━━━━━ 28s 126ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 3 - Accuracy: 0.99375000, Recall: 0.99375000, Precision: 0.99390244, F1 Score: 0.99376813, AUC: 0.99583333
161/161 ━━━━━━━━━━━━━━━━━━━━ 31s 134ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step
Run 4 - Accuracy: 0.99125000, Recall: 0.99125000, Precision: 0.99390244, F1 Score: 0.99249916, AUC: 0.99458333
161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 130ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step
Run 5 - Accuracy: 0.99250000, Recall: 0.99250000, Precision: 0.99390244, F1 Score: 0.99313524, AUC: 0.99520833
161/161 ━━━━━━━━━━━━━━━━━

# EfficientNetB3 + LBP + SVM Classifier

In [21]:
from sklearn.svm import SVC

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)
        
        model_FE_effnet = model_effnet()
        model_FE_effnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_effnet = model_FE_effnet.predict(X_train)
        test_feature_effnet = model_FE_effnet.predict(X_test)
        
        final_train = np.hstack((train_feature_effnet, train_feature_lbp))
        final_test = np.hstack((test_feature_effnet, test_feature_lbp))

        # Define and train SVM Classifier
        svm = SVC(probability=True)
        svm.fit(final_train, y_train_cat)
        test_pred = svm.predict(final_test)
        test_pred_proba = svm.predict_proba(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'EfficientNetB3+LBP+SVM.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 28s 125ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step
Run 1 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 27s 123ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 2 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 28s 127ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 3 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 36s 135ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step
Run 4 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 30s 131ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step
Run 5 - Accuracy: 0.99750000, Recall: 0.99750000, Precision: 0.99752475, F1 Score: 0.99749994, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━

# EfficientNetB3 + LBP + XGBoost Classifier

In [22]:
from xgboost import XGBClassifier

def train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test):
    with strategy.scope():  
        y_train_cat = convert_one_hot_to_categorical(y_train)
        y_test_cat = convert_one_hot_to_categorical(y_test)
        
        model_FE_effnet = model_effnet()
        model_FE_effnet.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_feature_effnet = model_FE_effnet.predict(X_train)
        test_feature_effnet = model_FE_effnet.predict(X_test)
        
        final_train = np.hstack((train_feature_effnet, train_feature_lbp))
        final_test = np.hstack((test_feature_effnet, test_feature_lbp))

        # Define and train XGB Classifier
        xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
        xgb.fit(final_train, y_train_cat)
        test_pred = xgb.predict(final_test)
        test_pred_proba = xgb.predict_proba(final_test)

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test_cat, test_pred)
        recall = recall_score(y_test_cat, test_pred, average='weighted')
        precision = precision_score(y_test_cat, test_pred, average='weighted')
        f1 = f1_score(y_test_cat, test_pred, average='weighted')
        auc = roc_auc_score(y_test_cat, test_pred_proba, multi_class='ovr', average='weighted')

        return accuracy, recall, precision, f1, auc

# Perform multiple runs and store the results
num_runs = 10
results = {'accuracy': [], 'recall': [], 'precision': [], 'f1': [], 'auc': []}

for i in range(num_runs):
    accuracy, recall, precision, f1, auc = train_fuse_and_evaluate_model(X_train, y_train, X_test, y_test)
    results['accuracy'].append(accuracy)
    results['recall'].append(recall)
    results['precision'].append(precision)
    results['f1'].append(f1)
    results['auc'].append(auc)
    print(f"Run {i+1} - Accuracy: {accuracy:.8f}, Recall: {recall:.8f}, Precision: {precision:.8f}, F1 Score: {f1:.8f}, AUC: {auc:.8f}")

# Compute average and standard deviation for each metric
average_metrics = {metric: np.mean(values) for metric, values in results.items()}
std_metrics = {metric: np.std(values) for metric, values in results.items()}

print("\nAverage Metrics:")
for metric, value in average_metrics.items():
    print(f"{metric.capitalize()}: {value:.8f} (std: {std_metrics[metric]:.8f})")

# Convert results to a pandas DataFrame
results_df = pd.DataFrame(results)

# Save results to an Excel file
output_file = 'EfficientNetB3+LBP+XGB.xlsx'
results_df.to_excel(output_file, index_label='Run')

print(f"\nMetrics results saved to '{output_file}'")


161/161 ━━━━━━━━━━━━━━━━━━━━ 31s 136ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 1 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 30s 134ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 2 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 130ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step
Run 3 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 132ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step
Run 4 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━━━━ 28s 126ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step
Run 5 - Accuracy: 1.00000000, Recall: 1.00000000, Precision: 1.00000000, F1 Score: 1.00000000, AUC: 1.00000000
161/161 ━━━━━━━━━━━━━━━━━